In [2]:
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import pandas as pd
from datetime import date
from CanaryFinancialCalculations import CanaryFinancialCalculations 
import cvxpy as cp


In [3]:
alpaca_api_key = 'PK306FA7SFOSMY9LBUZ4'
alpaca_secret_key = 'oNujbWMi3ZLrZuVggRN1DnZBLs38EJO00cwyOgsE'
base_url = 'https://paper-api.alpaca.markets'

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2")

tickers = ["AZO", "NFLX", "AMD", "BKNG"]
benchmark= ["SPY"]
timeframe = '1Day'
initial_investment = 10000
weights = [(1 / len(tickers))] * len(tickers)
weights_spy = [1]

start_date = pd.Timestamp("2018-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-01-01", tz="America/New_York").isoformat()

portfolio_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df


# User Portfolio

In [4]:
portfolio_df = portfolio_df.pivot(columns='symbol', values='close')
portfolio_df

symbol,AMD,AZO,BKNG,NFLX
timestamp,,,,
2018-01-02 05:00:00+00:00,10.98,736.54,1784.75,201.070
2018-01-03 05:00:00+00:00,11.55,749.43,1824.16,205.050
2018-01-04 05:00:00+00:00,12.12,761.26,1807.25,205.630
2018-01-05 05:00:00+00:00,11.88,775.50,1823.54,209.990
2018-01-08 05:00:00+00:00,12.28,766.48,1851.81,212.050
...,...,...,...,...
2022-12-23 05:00:00+00:00,64.52,2430.65,1971.52,294.965
2022-12-27 05:00:00+00:00,63.27,2454.32,1998.27,284.170
2022-12-28 05:00:00+00:00,62.57,2461.63,1958.84,276.880


In [5]:
portfolio_added = portfolio_df
portfolio_added = portfolio_added.dot(weights)
portfolio_added_df = pd.DataFrame(portfolio_added)
portfolio_added_df.columns = ['Portfolio']
portfolio_added_df.index = portfolio_added_df.index.date
portfolio_added_df.index.name='Date'
portfolio_added_df

,Portfolio
Date,
2018-01-02,683.33500
2018-01-03,697.54750
2018-01-04,696.56500
2018-01-05,705.22750
2018-01-08,710.65500
...,...
2022-12-23,1190.41375
2022-12-27,1200.00750
2022-12-28,1189.98000


In [6]:

portfolio_df_pct = portfolio_df.pct_change()
portfolio_df_pct.index = portfolio_df_pct.index.date
portfolio_df_pct = portfolio_df_pct.dropna()
portfolio_df_pct.index.name='Date'
portfolio_df_pct

symbol,AMD,AZO,BKNG,NFLX
Date,,,,
2018-01-03,0.051913,0.017501,0.022082,0.019794
2018-01-04,0.049351,0.015785,-0.009270,0.002829
2018-01-05,-0.019802,0.018706,0.009014,0.021203
2018-01-08,0.033670,-0.011631,0.015503,0.009810
2018-01-09,-0.037459,0.005832,0.007279,-0.012921
...,...,...,...,...
2022-12-23,0.010335,0.011145,0.019933,-0.009353
2022-12-27,-0.019374,0.009738,0.013568,-0.036598
2022-12-28,-0.011064,0.002978,-0.019732,-0.025654


In [7]:
user_returns = portfolio_df_pct.dot(weights)
user_returns_df = pd.DataFrame(user_returns)
user_returns_df.columns = ['Portfolio']
user_returns_df

,Portfolio
Date,
2018-01-03,0.027822
2018-01-04,0.014674
2018-01-05,0.007280
2018-01-08,0.011838
2018-01-09,-0.009317
...,...
2022-12-23,0.008015
2022-12-27,-0.008166
2022-12-28,-0.013368


# Benchmark Dataframe

In [8]:
benchmark_df = alpaca.get_bars(
    benchmark,
    timeframe,
    start=start_date,
    end=end_date
).df


In [9]:
benchmark_df = benchmark_df.pivot(columns='symbol', values='close')
benchmark_df_pct = benchmark_df.pct_change()
benchmark_df_pct.index = benchmark_df_pct.index.date
benchmark_df_pct = benchmark_df_pct.dropna()
benchmark_df_pct.index.name='Date'
benchmark_df_pct

symbol,SPY
Date,
2018-01-03,0.006325
2018-01-04,0.004067
2018-01-05,0.006739
2018-01-08,0.001865
2018-01-09,0.002264
...,...
2022-12-23,0.005752
2022-12-27,-0.003943
2022-12-28,-0.012428


In [10]:
benchmark_df.index = benchmark_df.index.date
benchmark_df.index.name='Date'
benchmark_df

symbol,SPY
Date,
2018-01-02,268.77
2018-01-03,270.47
2018-01-04,271.57
2018-01-05,273.40
2018-01-08,273.91
...,...
2022-12-23,382.91
2022-12-27,381.40
2022-12-28,376.66


# Combining Portfolio and SPY

In [11]:
comparing_portfolios = pd.concat([user_returns_df, benchmark_df_pct], axis=1, join='inner')
comparing_portfolios

,Portfolio,SPY
Date,,
2018-01-03,0.027822,0.006325
2018-01-04,0.014674,0.004067
2018-01-05,0.007280,0.006739
2018-01-08,0.011838,0.001865
2018-01-09,-0.009317,0.002264
...,...,...
2022-12-23,0.008015,0.005752
2022-12-27,-0.008166,-0.003943
2022-12-28,-0.013368,-0.012428


# Cumulative Returns

In [12]:
user_cumulative_returns = CanaryFinancialCalculations.cumulative_returns(user_returns_df, initial_investment)
user_cumulative_returns

,Profit
Date,
2018-01-03,10278.222352
2018-01-04,10429.041229
2018-01-05,10504.966412
2018-01-08,10629.323239
2018-01-09,10530.285729
...,...
2022-12-23,30434.741712
2022-12-27,30186.203601
2022-12-28,29782.681979


In [13]:
benchmark_returns = CanaryFinancialCalculations.cumulative_returns(benchmark_df_pct, initial_investment)
benchmark_returns

,Profit
Date,
2018-01-03,10063.251107
2018-01-04,10104.178294
2018-01-05,10172.266250
2018-01-08,10191.241582
2018-01-09,10214.309633
...,...
2022-12-23,14246.753730
2022-12-27,14190.571864
2022-12-28,14014.212896


# Covariance

In [14]:
user_covariance = CanaryFinancialCalculations.covariance(comparing_portfolios, "Portfolio", "SPY")
user_covariance

Date
2018-01-03         NaN
2018-01-04         NaN
2018-01-05         NaN
2018-01-08         NaN
2018-01-09         NaN
                ...   
2022-12-23    0.000219
2022-12-27    0.000220
2022-12-28    0.000219
2022-12-29    0.000248
2022-12-30    0.000166
Length: 1258, dtype: float64

# Variance

In [15]:
user_variance = CanaryFinancialCalculations.variance(comparing_portfolios, "SPY")
user_variance

Date
2018-01-03         NaN
2018-01-04         NaN
2018-01-05         NaN
2018-01-08         NaN
2018-01-09         NaN
                ...   
2022-12-23    0.000175
2022-12-27    0.000175
2022-12-28    0.000171
2022-12-29    0.000190
2022-12-30    0.000134
Name: SPY, Length: 1258, dtype: float64

# Beta

In [16]:
user_beta = CanaryFinancialCalculations.beta(user_covariance, user_variance)
user_beta

,Beta
Date,
2018-02-01,1.183638
2018-02-02,1.139588
2018-02-05,1.086309
2018-02-06,1.083101
2018-02-07,1.079957
...,...
2022-12-23,1.252632
2022-12-27,1.256542
2022-12-28,1.281794


# Drawdown

In [17]:
user_daily_drawdown = CanaryFinancialCalculations.daily_drawdown(portfolio_added_df)
user_daily_drawdown

,Portfolio
Date,
2018-01-02,0.00
2018-01-03,2.04
2018-01-04,2.04
2018-01-05,3.10
2018-01-08,3.84
...,...
2022-12-23,57.19
2022-12-27,57.19
2022-12-28,57.19


# Annual Returns

In [18]:
user_annual_returns = CanaryFinancialCalculations.annual_return(portfolio_added_df, 'Portfolio')
user_annual_returns

12.111524656294836

In [19]:
benchmark_annual_return = CanaryFinancialCalculations.annual_return(benchmark_df, 'SPY')
benchmark_annual_return

7.308526750062572

# Tracking Error

In [20]:
user_tracking_error = CanaryFinancialCalculations.tracking_error(user_annual_returns, benchmark_annual_return)
user_tracking_error

4.802997906232265

# Sharpe Ratio

In [21]:
user_sharpe_ratio = CanaryFinancialCalculations.sharpe_ratio(user_returns_df)
user_sharpe_ratio

,Sharpe Ratio
Portfolio,0.888538


# Return on Investment

In [22]:
user_roi = CanaryFinancialCalculations.return_on_investment(user_cumulative_returns, 'Profit', initial_investment)
user_roi

207.54956447097382

# Standard Deviation

In [23]:
user_std = CanaryFinancialCalculations.standard_deviation(user_returns_df)
user_std

,Portfolio
Date,
2018-02-01,0.012578
2018-02-02,0.013268
2018-02-05,0.016323
2018-02-06,0.016863
2018-02-07,0.016791
...,...
2022-12-23,0.018111
2022-12-27,0.018161
2022-12-28,0.018204


# Portfolio Distribution

In [24]:
user_portfolio_distribution = CanaryFinancialCalculations.portfolio_distribution_chart(tickers, weights)
user_portfolio_distribution